In [1]:
import pandas as pd
import numpy 
import regex as re
import random
import sklearn


import scipy



In [2]:
pheno = pd.read_table("pheno.txt")
data = pd.read_table("normalized_data.txt")

pheno.head()


,ParticipantID,Phenotype,Sample_Source,PlateId,Sex,HybControlNormScale,RowCheck,NormScale_20,NormScale_0_005,NormScale_0_5,ANMLFractionUsed_20,ANMLFractionUsed_0_005,ANMLFractionUsed_0_5
0,C1,Control,Blood,P0023923,F,0.947480,PASS,0.769958,0.813128,0.818578,0.868,0.838,0.745
1,C2,Control,Blood,P0023895,F,0.891341,PASS,0.765640,0.967612,0.891220,0.874,0.900,0.790
2,C3,Control,Blood,P0023895,F,0.888196,PASS,0.412029,0.918688,0.810701,0.732,0.881,0.666
3,C4,Control,Blood,P0023923,F,0.856026,PASS,0.774548,0.866460,0.810048,0.879,0.875,0.777
4,C5,Control,Blood,P0023895,F,0.982222,PASS,0.665371,0.833492,0.820785,0.804,0.900,0.852


In [3]:
data.head()

metadata = data.iloc[:, :4]

metadata

,Molecule,Protein,Gene,UniProt
0,10000-28,CRBB2,CRYBB2,P43320
1,10001-7,c-Raf,RAF1,P04049
2,10003-15,ZNF41,ZNF41,P51814
3,10006-25,ELK1,ELK1,P19419
4,10008-43,GUC1A,GUCA1A,P43080
...,...,...,...,...
5279,9993-11,ZN264,ZNF264,O43296
5280,9994-217,ATP4B,ATP4B,P51164
5281,9995-6,DUT,DUT,P33316
5282,9997-12,UBXN4,UBXN4,Q92575


In [4]:
data = data.transpose()



In [5]:
data.columns = data.iloc[0]
data.drop(data.index[0:4], inplace=True)





In [6]:
data = data.apply(pd.to_numeric)

In [7]:
data.dtypes

Molecule
10000-28    float64
10001-7     float64
10003-15    float64
10006-25    float64
10008-43    float64
             ...   
9993-11     float64
9994-217    float64
9995-6      float64
9997-12     float64
9999-1      float64
Length: 5284, dtype: object

In [8]:
target = data.index.tolist()

#replace controls with 0 and patient with 1

target = [0 if re.search('[C]', x) else 1 for x in target ]


In [11]:
data["target"] = target
data['target']

C1     0
C2     0
C3     0
C4     0
C5     0
C8     0
C9     0
C11    0
C14    0
C15    0
C16    0
C17    0
C18    0
C19    0
C20    0
C21    0
C24    0
C25    0
C26    0
C27    0
P1     1
P2     1
P4     1
P6     1
P9     1
P11    1
P13    1
P17    1
P18    1
P19    1
P20    1
P21    1
P24    1
P25    1
P27    1
P28    1
P29    1
P30    1
P31    1
P32    1
Name: target, dtype: int64

## Feature Selection

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [14]:
X = data.drop(["target"],axis=1)
y = data['target']

#Using Pearson Correlation
#plt.figure()
cor = data.corr()


In [15]:
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
#plt.show()

#Correlation with output variable
cor_target = abs(cor["target"])#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]


Molecule
10440-26     0.510134
11696-7      0.554680
12800-5      0.524439
13463-1      0.521459
14008-22     0.503613
14054-17     0.507983
14131-37     0.512208
16915-153    0.530606
18819-21     0.545189
18871-24     0.621119
2590-69      0.579863
2614-28      0.574972
2615-60      0.520315
2654-19      0.570139
2944-66      0.516414
3073-51      0.576326
4374-45      0.500888
5134-52      0.596003
6375-75      0.503063
7776-20      0.527375
8483-5       0.511841
target       1.000000
Name: target, dtype: float64

In [26]:
rel_data = data[[x for x in relevant_features.index]]
rel_data

Molecule,10440-26,11696-7,12800-5,13463-1,14008-22,14054-17,14131-37,16915-153,18819-21,18871-24,...,2615-60,2654-19,2944-66,3073-51,4374-45,5134-52,6375-75,7776-20,8483-5,target
C1,1012.6,10152.1,4794.6,1210.1,1006.0,7789.0,5362.8,11469.4,3487.7,1296.5,...,2922.8,13382.9,1181.7,4035.6,16061.5,2704.2,2197.5,8136.8,96.9,0
C2,870.8,8245.5,4726.7,970.7,1070.2,6609.3,4890.4,8125.7,6824.6,1330.0,...,2735.6,9179.1,1081.5,4357.8,11091.8,2488.9,1980.3,9561.3,107.2,0
C3,953.5,6604.4,2119.7,1105.6,791.1,3910.2,3518.1,11253.5,7253.6,996.2,...,1723.7,6556.5,1047.1,3326.8,7473.5,1791.7,1651.5,4944.6,118.8,0
C4,1075.5,7772.7,4769.7,1240.0,1002.7,7868.4,5953.9,10975.6,6196.9,1521.1,...,2924.8,12731.9,1066.6,5208.8,15619.9,3511.5,2047.3,8012.1,99.6,0
C5,1181.2,10094.5,4297.4,1232.1,1039.6,7901.5,5596.9,8758.8,7051.5,1461.0,...,2818.8,11596.8,1174.2,5338.6,12462.0,3565.5,2029.0,9353.2,103.0,0
C8,998.7,8086.3,4106.6,1175.8,1027.9,6392.8,5112.2,8576.8,7742.0,1356.8,...,2782.7,10338.4,1123.6,4355.2,12649.9,2729.3,2198.3,8744.2,125.9,0
C9,1105.4,8161.1,3687.1,1763.6,909.6,5981.2,6162.2,13312.2,8168.4,1410.4,...,2953.0,11457.2,1212.3,4593.6,14561.6,2887.0,2058.3,8128.1,112.1,0
C11,1092.0,6620.6,3173.5,1029.7,1011.7,7584.5,4616.8,11967.0,7676.9,1339.6,...,2751.3,9690.3,1208.4,3977.8,9395.5,2846.5,2229.1,8320.1,92.9,0
C14,1050.7,10211.3,4660.2,1236.4,1044.3,6115.0,6344.0,8130.5,6712.1,1350.9,...,2949.2,10917.2,1283.6,4135.3,11969.9,2664.3,2263.2,9317.4,107.9,0
C15,1056.7,9012.6,4125.1,1122.5,1238.2,7585.5,7002.7,10619.4,8205.1,1386.5,...,3554.6,11095.9,1354.8,4228.1,15890.3,2717.6,2585.7,10966.9,97.1,0


### Fitting GLM

In [37]:
from sklearn.linear_model import LogisticRegression

X = data.drop(["target"],axis=1)
y = data['target']

clf = LogisticRegression()
clf.fit(X,y)

clf.coef_

array([[5.24671850e-05, 1.23688370e-07, 4.24787073e-08, ...,
        1.84657639e-05, 1.18166631e-05, 2.05128490e-06]])

In [9]:
cases = data[data["target"]=="Case"]
controls = data[data["target"]!="Case"]


def simulate_protein_data(df):
    means = df.mean(axis=0).tolist()
    sds = df.std(axis=0).tolist()

    if len(means) != len(sds): ## sanity check
        return "List Imbalance"
    
    simulated_data = []
    for i in range(len(means)):
        col = []
        for _ in range((1000)):
            col.append(random.gauss(means[i],sds[i]))
        simulated_data.append(col)
        

    simulated_dataframe = pd.DataFrame(simulated_data)
    return simulated_dataframe 




In [10]:
cases_simul = simulate_protein_data(cases)

/var/folders/yj/znfgbkgj4f1gvy__jfl3rw6c0000gn/T/ipykernel_4562/336261786.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  means = df.mean(axis=0).tolist()
/var/folders/yj/znfgbkgj4f1gvy__jfl3rw6c0000gn/T/ipykernel_4562/336261786.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  sds = df.std(axis=0).tolist()


In [113]:
cases_simul

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,8334.217918,15072.688985,3922.138778,1633.791290,-2590.013169,7053.105447,-1539.653744,-3311.676133,11819.928607,-16755.456791,...,6014.038696,-1443.315256,3161.517565,15753.056642,5576.299950,5475.473060,12280.039414,14103.681075,344.288393,324.798358
1,248.706004,267.486169,272.785676,287.504725,173.241579,216.558338,228.824812,211.371410,223.870291,208.043209,...,220.121159,225.676685,212.704080,244.827548,168.954258,242.559165,285.722037,200.842513,235.087296,226.222470
2,98.585963,147.262624,76.114877,128.766304,112.747871,155.527671,126.142614,-33.501536,-23.838260,-128.721032,...,283.768349,224.636131,57.799425,381.234330,-148.638721,136.141746,269.711737,232.969882,295.944678,229.453387
3,1317.570612,1975.024242,129.341223,142.268494,699.371529,537.653463,1198.054640,828.036901,1505.475510,1120.426955,...,1768.038110,1531.242452,500.466676,1344.665332,1179.482671,515.451689,863.797658,733.972598,1088.382014,1512.896956
4,432.963741,472.295452,408.891090,529.144088,475.280567,615.141049,463.589668,566.451140,535.257246,596.863478,...,366.506273,542.818530,600.169346,521.841979,487.522118,504.500491,531.083260,502.997161,493.142163,567.518730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,943.144158,925.148804,999.098764,936.640762,975.261623,939.241526,954.242762,1015.618551,906.042185,1028.604293,...,1001.793540,1027.977605,969.866723,902.185022,936.877349,1001.894603,901.265342,1039.149707,1010.187522,1073.791850
5280,8962.379246,9122.403800,13154.452786,10914.819767,115.068199,14535.323578,16643.952881,8238.136815,3454.127795,1341.242917,...,6423.377584,13498.819835,12026.845140,629.840518,-1154.038115,5666.387960,10853.943206,4019.609000,16629.708065,4459.612843
5281,1239.332904,5428.903803,4636.139394,3412.581616,-4276.382197,-2462.182266,168.439642,5547.735757,2854.296669,2354.556349,...,5410.451716,-1005.540574,2596.290598,5802.001753,2530.711202,4332.907089,1886.677745,2249.585962,7027.182679,8128.055261
5282,16870.038273,5562.826465,2438.880935,10031.513966,11031.770788,5391.403300,10224.815110,7821.346307,7477.350676,8827.703978,...,9370.152799,7714.995680,20964.942899,6122.097657,19967.204201,5555.968158,15366.061792,11142.898122,13540.371085,2856.684355


In [100]:
cases.mean(axis=0).tolist()

/var/folders/yj/znfgbkgj4f1gvy__jfl3rw6c0000gn/T/ipykernel_57915/2859408631.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cases.mean(axis=0).tolist()[0]


2587.92

 ## Data Simulation Time

In [85]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data.drop(['target'], axis = 1), data["target"] ,test_size=0.25,random_state=0)

In [86]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Load the dataset

# Create a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [87]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[1, 4],
       [1, 4]])

array(['Control', 'Control', 'Case', 'Case', 'Control', 'Control',
       'Control', 'Control', 'Control', 'Control'], dtype=object)